In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langid

In [6]:
# Importing libraries
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from textblob import Word, TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings
warnings.filterwarnings('ignore')
import re
import zipfile
import concurrent.futures
import langid

In [5]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('vader_lexicon', quiet=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [32]:
# read data from zipfile
data_subset_Azganush = pd.read_csv('/content/drive/MyDrive/Omdena/Project_1/data_subset_Azganush.csv')
data_subset_Amal = pd.read_csv('/content/drive/MyDrive/Omdena/Project_1/data_subset_Amal.csv')
data_subset_Felix = pd.read_csv('/content/drive/MyDrive/Omdena/Project_1/data_subset_Felix.csv')
data_subset_Royston = pd.read_csv('/content/drive/MyDrive/Omdena/Project_1/data_subset_Royston.csv')
data_subset_Sanjiban = pd.read_csv('/content/drive/MyDrive/Omdena/Project_1/data_subset_Sanjiban.csv')

# show sample
data_subset_Azganush.head()

,site,role,company_name,location,job_description,salary_range,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,glassdoor,Analyst – NSE - BSO - UX and Dev,Deloitte,Hyderābād,Analyst – UX and Dev – Deloitte Support Servic...,₹6L - ₹8L (Glassdoor Est.),4.1 ★,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,glassdoor,FP&A Analyst,Eastvantage,Bengaluru,We are looking for a FP&A Analyst to join the ...,₹7L - ₹13L (Employer Est.),4.3 ★,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,glassdoor,Analyst (International ),Disney,Bengaluru,Job Summary-\nWe are seeking a highly motivate...,"In this role, you’ll be working closely with t...",3.8 ★,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,glassdoor,Consultant - Business Analyst,Affine,Chennai,Job Description\nComplete accountability for d...,₹3L - ₹4L (Glassdoor Est.),4.8 ★,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glassdoor,Consultant - Business Analyst,Affine,Chennai,Minimun 5 to 6 years' experience in SQL query ...,₹3L - ₹4L (Glassdoor Est.),3.8 ★,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
def text_preprocessing(text, result = 'clean_tokens'):
    """
    Preprocesses the raw text applying the following steps: convert to string, remove newline characters, standardize, remove numbers, punctuation and stopwords, stem and lemmatize the words.

    Parameters:
    - text (str): The raw text to be preprocessed.
    - result (str): The step to include in the output. Possible values: 'text', 'combined_text', 'standardized_text', 'no_punctuation',
                                                                        'no_numbers', 'no_stopwords', 'tokens', 'stemmed_tokens', 'lemmatized_tokens', 'clean_tokens'.
                    Default is 'clean_tokens'.

    Returns:
    - str or list: The result of the specified step applied.
    """
    text = str(text)
    if result == 'text':
      return text
    # remove newline characters
    combined_text = text.replace('\n', ' ')
    if result == 'combined_text':
      return combined_text

    # standardization of letters (make lowercase)
    standardized_text = combined_text.lower()
    if result == 'standardized_text':
      return standardized_text

    # remove punctuation
    no_punctuation = re.sub(r'[^\w\s]', '', standardized_text)
    if result == 'no_punctuation':
      return no_punctuation

    # remove numbers
    no_numbers = re.sub(r'\d', '', no_punctuation)
    if result == 'no_numbers':
      return no_numbers

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    no_stopwords = " ".join([word for word in no_numbers.split() if word not in stop_words])
    if result == 'no_stopwords':
      return no_stopwords

    # spacy tokenization
    nlp = spacy.load('en_core_web_sm')
    doc_tokenize = nlp(no_stopwords)
    tokens = [token.text for token in doc_tokenize]
    if result == 'tokens':
      return tokens

    # stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    if result == 'stemmed_tokens':
      return stemmed_tokens

    # lemmatization
    sentence = " ".join(tokens)
    doc_lemmitize = nlp(sentence)
    lemmatized_tokens = [token.lemma_ for token in doc_lemmitize]
    if result == 'lemmatized_tokens':
      return lemmatized_tokens

    # final cleaning: remove empty strings, single letters and duplicates
    clean_tokens = [token for token in list(set(lemmatized_tokens)) if token.strip() != '' and len(token) > 1]
    if result == 'clean_tokens':
      return clean_tokens

def is_eng(text):
    """
    Checks if the input text is in the English language.

    Parameters:
    - text (str): The input text to be checked for English language.

    Returns:
    - bool: True if the text is identified as English, False otherwise.
    """
    text = str(text)
    # Detects the language of the text
    lang, confidence = langid.classify(str(text))

    # Check if the language is English
    return lang == 'en'

def parallel_processing(iterable, processing_function, *processing_args, workers=3):
    """
    Process elements in parallel using ThreadPoolExecutor.

    Parameters:
    - iterable (iterable): The iterable containing elements to be processed.
    - processing_function (function): The processing function to apply to the elements.
    - processing_args: Additional arguments to pass to the processing function.
    - workers (int, optional): The number of parallel workers. Default is 3.

    Returns:
    - list: A list of processed elements after applying the specified processing function in parallel.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        processed_elements = list(executor.map(lambda element: processing_function(element, *processing_args), iterable))

    return processed_elements

exchange_rates = {
    'United Kingdom': {'currency_name': 'Pounds Sterling GBP', 'currency_symbol': '£', 'exchange_rate': 0.79},
    'United States': {'currency_name': 'US Dollar USD', 'currency_symbol': '$', 'exchange_rate': 1.00},
    'Eurozone': {'currency_name': 'Euro EUR', 'currency_symbol': '€', 'exchange_rate': 0.92},
    'India': {'currency_name': 'Indian Rupee INR', 'currency_symbol': '₹', 'exchange_rate': 83.30}
}

def is_valid_currency_symbol(symbol, exchange_rates = exchange_rates):
    """
    Check if a given currency symbol is present in the exchange rates dictionary.

    Parameters:
    - symbol (str): The currency symbol to check.
    - exchange_rates (dict): Dictionary containing currency information.

    Returns:
    - bool: True if the symbol is present, False otherwise.
    """
    for currency_info in exchange_rates.values():
        if 'currency_symbol' in currency_info and currency_info['currency_symbol'] == symbol:
            return True
    return False

import re

def extract_salary(text, exchange_rates = exchange_rates):
    """
    Extracts salary information from a given text.

    Parameters:
    - text (str): The input text containing salary information.
    - exchange_rates (dict): Dictionary containing currency information.

    Returns:
    - tuple or None: A tuple containing (min_salary_usd, max_salary_usd) with the extracted
      minimum and maximum salary values in USD. Returns (None, None) if no valid
      salary information is found.
    """
    text = str(text)
    # Define the pattern for salary information
    get_salary_pattern = re.compile(r'([\$€£₹])\s*([\d,]+[a-zA-Z]?)\s*-\s*([\$€£₹])?\s*([\d,]+[a-zA-Z]?)?')

    # Define the pattern for extracting numeric values with optional suffix
    get_number_pattern = re.compile(r'\b(\d+(?:,\d{3})*(?:\.\d+)?)(?:[a-zA-Z]+)?\b')

    # Define the short scale mapping for suffixes
    short_scale_mapping = {'K': 1e3, 'M': 1e6, 'B': 1e9, 'T': 1e12, 'L': 1e5}

    # Search for the salary pattern in the text
    match = get_salary_pattern.search(text)

    if match:
        # Extract relevant information from the matched groups
        min_currency_symbol = match.group(1)
        min_salary_str = match.group(1) + match.group(2)
        max_salary_str = match.group(3) + match.group(4) if match.group(3) else min_salary_str

        try:
            # Extract numeric values with optional suffix
            min_salary = float(get_number_pattern.search(min_salary_str).group(1).replace(',', ''))
            max_salary = float(get_number_pattern.search(max_salary_str).group(1).replace(',', ''))

            # Check if there's a suffix and apply the multiplier if needed
            suffix = min_salary_str[-1]
            if suffix.upper() in short_scale_mapping:
                min_salary *= short_scale_mapping[suffix.upper()]
                max_salary *= short_scale_mapping[suffix.upper()]
            elif not suffix.isdigit() and len(suffix) >= 1:
                print('Unknown suffix in the salary range: ' + str(suffix) + ' Update short_scale_mapping dictionary.')

        except (ValueError, AttributeError):
            return (None, None)

        # Check if the currency symbol is valid
        if not is_valid_currency_symbol(min_currency_symbol):
            print('Unknown currency symbol: ' + str(min_currency_symbol) + ' Update exchange_rates dictionary and get_salary_pattern.')
            print('Available currency symbols:', list(exchange_rates.keys()))
            return (None, None)

        # Find currency information based on the symbol
        currency_info = next((info for info in exchange_rates.values() if info['currency_symbol'] == min_currency_symbol), None)

        if currency_info:
            # Convert salaries to USD using exchange rate
            exchange_rate = currency_info['exchange_rate']
            min_salary_usd = round(min_salary / exchange_rate)
            max_salary_usd = round(max_salary / exchange_rate)

            return (min_salary_usd, max_salary_usd)
        else:
            return (None, None)
    else:
        return (None, None)

def extract_job_level(text):
    """
    Extracts job level from a given text.

    Parameters:
    - text (str): The input text.

    Returns:
    - str or None: The extracted job level string. Returns None if no job level is found.
    """
    text = str(text)
    # Define keywords related to job levels
    job_level_keywords = {
        'entry-level': 'entry',
        'junior': 'junior',
        'mid-level': 'mid',
        'senior': 'senior'

    }

    # Create a regular expression pattern to match job level keywords
    pattern = re.compile(r'\b(?:' + '|'.join(re.escape(keyword) for keyword in job_level_keywords.keys()) + r')\b', flags=re.IGNORECASE)

    # Search for job level keywords in the job description
    match = pattern.search(text)

    if match:
        # Return the matched job level string
        return job_level_keywords[match.group().lower()]
    else:
        return None

def process_jobs_df(df):
  """
    Process jobs dataset.

    Parameters:
    - df (pd.DataFrame): Input DataFrame.

    Returns:
    - pd.DataFrame: Processed DataFrame.
  """
  columns = ['site', 'role', 'company_name', 'location', 'job_description',
            'salary_range', 'ratings', 'job_level', 'Search_Keywords',
            'Search_Location', 'Job_Detail_Link', 'Post_Time', 'Current_Time',
            'job_type']
  # drop duolicates
  df.drop_duplicates(inplace = True)

  # remove non-english rows (based on job_description)
  eng_lang = df['job_description'].apply(is_eng)
  df = df[eng_lang]

  # remove senior roles
  df = df[~(df['role'].str.lower().str.contains('sr') | df['role'].str.lower().str.contains('senior'))]

  # process 'site' column: convert to lowercase
  processed_site = df['site'].apply(lambda x: text_preprocessing(x, 'standardized_text'))

  # process 'role', 'company_name', 'location', 'job_description', columns with the following stes:
  # 1)convert to string, 2)remove newline chars, 3) make lowercase, 4) remove punctuation, 5) remove numbers 6) remove stopwords
  processed_role = parallel_processing(df['role'], text_preprocessing, ('no_stopwords'))
  processed_company_name = parallel_processing(df['company_name'], text_preprocessing, 'no_stopwords')
  processed_location = parallel_processing(df['location'], text_preprocessing, 'no_stopwords')
  processed_job_description = parallel_processing(df['job_description'], text_preprocessing, 'no_stopwords')

  # process 'salary_range' column: extract salary (range or number), convert the number into float, convert to USD using the exchange_rates dictionary,
  # add minimum and maximum values in two different columns
  processed_min_salary_usd = list(map(lambda x: x[0], parallel_processing(df['salary_range'], extract_salary)))
  processed_max_salary_usd = list(map(lambda x: x[1], parallel_processing(df['salary_range'], extract_salary)))

  # process 'ratings' column: remove start symbol
  processed_ratings = parallel_processing(df['ratings'], lambda x: str(x).replace(' ★', ''))

  # process 'job_level' column: get job_level in lowercase from job_level column, if it's get job description from role column, otherwise from job_description
  # by targeting the following keywords: 'entry', 'junior', 'mid_level', 'senior'
  processed_job_level = df.apply(lambda row: row['job_level'] if pd.notnull(row['job_level'])
                                                              else extract_job_level(row['role']) if pd.notnull(extract_job_level(row['role']))
                                                              else extract_job_level(row['job_description']), axis=1)

  # process 'Search_Keywords', 'Search_Location' and 'job_type' columns: make lowercase & remove punctuation
  processed_Search_Keywords = parallel_processing(df['Search_Keywords'], text_preprocessing, ('no_punctuation'))
  processed_Search_Location = parallel_processing(df['Search_Location'], text_preprocessing, ('no_punctuation'))
  processed_job_type = parallel_processing(df['job_type'], text_preprocessing, ('no_punctuation'))

  # process 'Job_Detail_Link' function: just convert to string
  processed_Job_Detail_Link = parallel_processing(df['Job_Detail_Link'], text_preprocessing, ('text'))

  # process 'Post_Time', 'Current_Time' columns: make lowercase
  processed_Post_Time = parallel_processing(df['Post_Time'], text_preprocessing, ('standardized_text'))
  processed_Current_Time = parallel_processing(df['Current_Time'], text_preprocessing, ('standardized_text'))

  processed_df = pd.DataFrame({
        'site': processed_site,
        'role': processed_role,
        'company_name': processed_company_name,
        'location': processed_location,
        'job_description': processed_job_description,
        'min_salary': processed_min_salary_usd,
        'max_salary': processed_max_salary_usd,
        'ratings': processed_ratings,
        'job_level': processed_job_level,
        'Search_Keywords': processed_Search_Keywords,
        'Search_Location': processed_Search_Location,
        'Job_Detail_Link': processed_Job_Detail_Link,
        'Post_Time': processed_Post_Time,
        'Current_Time': processed_Current_Time,
        'job_type': processed_job_type
    })
  return processed_df

In [39]:
processed_df_Azganush = process_jobs_df(data_subset_Azganush)
print('lost data: ' + str(processed_df_Azganush.shape[0] - data_subset_Azganush.shape[0]) + ' rows')
processed_df_Azganush.head()

lost data: -361 rows


,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,glassdoor,analyst nse bso ux dev,deloitte,hyderābād,analyst ux dev deloitte support services india...,7203.0,9604.0,4.1,None,nan,nan,nan,nan,nan,nan
1,glassdoor,fpa analyst,eastvantage,bengaluru,looking fpa analyst join team help clients bud...,8403.0,15606.0,4.3,None,nan,nan,nan,nan,nan,nan
2,glassdoor,analyst international,disney,bengaluru,job summary seeking highly motivated talented ...,NaN,NaN,3.8,senior,nan,nan,nan,nan,nan,nan
3,glassdoor,consultant business analyst,affine,chennai,job description complete accountability delive...,3601.0,4802.0,4.8,senior,nan,nan,nan,nan,nan,nan
4,glassdoor,consultant business analyst,affine,chennai,minimun years experience sql query writing pos...,3601.0,4802.0,3.8,senior,nan,nan,nan,nan,nan,nan


In [40]:
processed_df_Amal = process_jobs_df(data_subset_Amal)
print('lost data: ' + str(processed_df_Amal.shape[0] - data_subset_Amal.shape[0]) + ' rows')
processed_df_Amal.head()

lost data: -886 rows


,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,glassdoor,industry ai hpc business development represent...,athonet,winnersh england,role designated onsite expectation primarily w...,24051.0,34177.0,nan,None,nan,nan,nan,nan,nan,nan
1,glassdoor,manager bb marketing analytics viator,tripadvisor,london england,believe better together tripadvisor welcome wo...,58228.0,69620.0,nan,None,nan,nan,nan,nan,nan,nan
2,glassdoor,global data analytics solutions expert,campari group,london england,date nov location london eng gb additional loc...,58228.0,105063.0,nan,senior,nan,nan,nan,nan,nan,nan
3,glassdoor,web analytics manager,orchard agency,salford north west england england,k manchesterhybrid st hybrid working days holi...,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan
4,glassdoor,category data analytics executive,better placed,leeds england,sector fmcg contact leah ross contact email le...,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan


In [41]:
processed_df_Felix = process_jobs_df(data_subset_Felix)
print('lost data: ' + str(processed_df_Felix.shape[0] - data_subset_Felix.shape[0]) + ' rows')
processed_df_Felix.head()

lost data: -278 rows


,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
1,indeed,data scientist,intone networks,remote bentonville ar,nan,NaN,NaN,nan,None,data scientist,united states,https://www.indeed.com/rc/clk?jk=a379fc6552f8a...,posted posted 30+ days ago,2023-11-14,nan
2,indeed,data scientist,centers disease control prevention,atlanta ga,nan,NaN,NaN,nan,None,data scientist,united states,https://www.indeed.com/rc/clk?jk=190a082079ea8...,posted posted 30+ days ago,2023-11-14,nan
4,indeed,analytics software engineer data scientist,headway cooperative,hybrid remote denver co,nan,NaN,NaN,nan,None,data scientist,united states,https://www.indeed.com/rc/clk?jk=e0500ffe1bdeb...,employer active 6 days ago,2023-11-14,nan
5,indeed,data scientist,takeda pharmaceutical,cambridge,nan,NaN,NaN,nan,None,data scientist,united states,https://www.indeed.com/rc/clk?jk=f785c9c503255...,posted posted 30+ days ago,2023-11-14,nan
6,indeed,data engineer,caci,fort liberty nc,nan,NaN,NaN,nan,None,data scientist,united states,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,posted posted 30+ days ago,2023-11-14,nan


In [43]:
processed_df_Royston = process_jobs_df(data_subset_Royston)
print('lost data: ' + str(processed_df_Royston.shape[0] - data_subset_Royston.shape[0]) + ' rows')
processed_df_Royston.head()

lost data: -123 rows


,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,indeed,conflicts analyst assistant,hogan lovells,johannesburg gauteng,keen become part truly global collaborative te...,None,None,nan,None,nan,nan,nan,nan,nan,fulltime
1,indeed,analyst property finance corporate,nedbank,johannesburg gauteng,job purpose assist executing implementing tran...,None,None,nan,None,nan,nan,nan,nan,nan,not available
4,indeed,finance business analyst,momentum investments,centurion gauteng,introduction clientfacing brands metropolitan ...,None,None,nan,None,nan,nan,nan,nan,nan,temporary
5,indeed,associate director valuations,deloitte,midrand gauteng,company description deloitte largest private p...,None,None,nan,junior,nan,nan,nan,nan,nan,fulltime
6,indeed,data analyst,panagora group,remote pretoria gauteng,job information projectactivity usaidsouth afr...,None,None,nan,senior,nan,nan,nan,nan,nan,not available


In [53]:
processed_df_Sanjiban = process_jobs_df(data_subset_Sanjiban)
print('lost data: ' + str(processed_df_Sanjiban.shape[0] - data_subset_Sanjiban.shape[0]) + ' rows')
processed_df_Sanjiban.head()

lost data: -613 rows


,site,role,company_name,location,job_description,min_salary,max_salary,ratings,job_level,Search_Keywords,Search_Location,Job_Detail_Link,Post_Time,Current_Time,job_type
0,glassdoor,principal data scientist silico antibody design,genmab,utrecht,genmab committed building extranotordinary fut...,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan
3,glassdoor,phd position mechanisms sensory perception cortex,vrije universiteit amsterdam,amsterdam,location amsterdam fte job description cortica...,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan
4,glassdoor,data engineer spark python airflow glue aws,apollo,netherlands,introductie looking experienced freelance seni...,NaN,NaN,nan,senior,nan,nan,nan,nan,nan,nan
6,glassdoor,technical domain architect dataai,tata steel,ijmuiden,nan,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan
8,glassdoor,group data engineer,stage entertainment,amsterdam,nan,NaN,NaN,nan,None,nan,nan,nan,nan,nan,nan


In [56]:
processed_df_Azganush.to_csv('data_subset_Azganush_procesed.csv')
processed_df_Amal.to_csv('data_subset_Amal_procesed.csv')
processed_df_Felix.to_csv('data_subset_Felix_procesed.csv')
processed_df_Royston.to_csv('data_subset_Royston_procesed.csv')
processed_df_Sanjiban.to_csv('data_subset_Sanjiban_procesed.csv')